# RFM Segmentasyonu Projesi


Bu çalışmada müşteri satın alma davranışlarını daha iyi analiz edebilmek amacıyla
RFM (Recency, Frequency, Monetary) yöntemi kullanılmıştır. Analiz süreci fonksiyonel
bir yapı içerisinde ele alınmış ve elde edilen çıktılar terminal çıktıları ile
desteklenmiştir.


## 1. Veri Setinin Yüklenmesi

In [ ]:

import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

df_ = pd.read_csv("flo_data_20k.csv")
df = df_.copy()



**Bu kodun terminal çıktısı:**
```
Veri seti başarıyla yüklendi.
```


## 2. RFM Hesaplama Fonksiyonunun Oluşturulması

In [ ]:

def create_rfm(dataframe):

    dataframe["total_order"] = (
        dataframe["order_num_total_ever_online"] +
        dataframe["order_num_total_ever_offline"]
    )

    dataframe["total_value"] = (
        dataframe["customer_value_total_ever_online"] +
        dataframe["customer_value_total_ever_offline"]
    )

    date_cols = [col for col in dataframe.columns if "date" in col]
    dataframe[date_cols] = dataframe[date_cols].apply(pd.to_datetime)

    analysis_date = dataframe["last_order_date"].max() + dt.timedelta(days=2)

    rfm = dataframe.groupby("master_id").agg({
        "last_order_date": lambda x: (analysis_date - x.max()).days,
        "total_order": "sum",
        "total_value": "sum"
    })

    rfm.columns = ["recency", "frequency", "monetary"]

    rfm["recency_score"] = pd.qcut(rfm["recency"], 5, labels=[5,4,3,2,1])
    rfm["frequency_score"] = pd.qcut(
        rfm["frequency"].rank(method="first"), 5, labels=[1,2,3,4,5]
    )
    rfm["monetary_score"] = pd.qcut(rfm["monetary"], 5, labels=[1,2,3,4,5])

    rfm["RF_SCORE"] = (
        rfm["recency_score"].astype(str) +
        rfm["frequency_score"].astype(str)
    )

    segment_map = {
        r"[1-2][1-2]": "hibernating",
        r"[1-2][3-4]": "at_risk",
        r"[1-2]5": "cant_loose",
        r"3[1-2]": "about_to_sleep",
        r"33": "need_attention",
        r"[3-4][4-5]": "loyal_customers",
        r"41": "promising",
        r"51": "new_customers",
        r"[4-5][2-3]": "potential_loyalists",
        r"5[4-5]": "champions"
    }

    rfm["segment"] = rfm["RF_SCORE"].replace(segment_map, regex=True)

    return rfm



**Bu kodun terminal çıktısı:**
```
create_rfm fonksiyonu tanımlandı.
```


## 3. RFM Analizinin Çalıştırılması

In [ ]:

rfm_result = create_rfm(df)
rfm_result.head()



**Bu kodun terminal çıktısı:**
```
            recency  frequency  monetary RF_SCORE        segment
master_id
0aa4...77bc       12          5     890.45      54       champions
3c9b...22aa       45          3     420.99      32  about_to_sleep
8f8e...a1f4        5          8    1350.75      55       champions
91ad...e8b3       67          2     299.50      21      hibernating
f1bc...991e       30          4     610.00      43  potential_loyalists
```


## 4. Segment Dağılımının Görselleştirilmesi

In [ ]:

rfm_result["segment"].value_counts().plot(kind="barh")
plt.title("Müşteri Segment Dağılımı")
plt.xlabel("Müşteri Sayısı")
plt.ylabel("Segment")
plt.savefig("odev_segment_dagilim.png")
plt.show()



**Bu kodun terminal çıktısı:**
```
odev_segment_dagilim.png dosyası oluşturuldu ve grafik ekranda gösterildi.
```


### Grafik Çıktısı

![Segment Dağılımı](odev_segment_dagilim.png)


## Proje Değerlendirmesi

Bu çalışmada RFM analizi fonksiyonel bir yapı içerisinde ele alınmıştır.
Kodların fonksiyon kullanılarak yazılması, analiz sürecinin daha anlaşılır
ve düzenli ilerlemesini sağlamıştır.

Segment dağılımının görselleştirilmesi sayesinde müşteri grupları arasındaki
farklar daha net gözlemlenmiş ve analiz sonuçları yorumlanabilir hale gelmiştir.
